# Accessing Databases via Web APIs: Lecture Code
* * * * *

In [5]:
# Import required libraries
import requests
from urllib import quote_plus
import json
from __future__ import division
import math

## 1. Constructing API GET Request

In the first place, we know that every call will require us to provide a) a base URL for the API, b) some authorization code or key, and c) a format for the response. So let's put store those in some variables.

In [6]:
# set key
key="ef9055ba947dd842effe0ecf5e338af9:15:72340235"

# set base url
base_url="http://api.nytimes.com/svc/search/v2/articlesearch"

# set response format
response_format=".json"

You often want to send some sort of data in the URL’s query string. This data tells the API what information you want. In our case, we want articles about Duke Ellington. Requests allows you to provide these arguments as a dictionary, using the `params` keyword argument. In addition to the search term `q`, we have to put in the `api-key` term.

In [7]:
# set search parameters
search_params = {"q":"Duke Ellington",
                 "api-key":key}       

Now we're ready to make the request. We use the `.get` method from the `requests` library to make an HTTP GET Request.

In [8]:
# make request
r = requests.get(base_url+response_format, params=search_params)

Now, we have a [response](http://docs.python-requests.org/en/latest/api/#requests.Response) object called `r`. We can get all the information we need from this object. For instance, we can see that the URL has been correctly encoded by printing the URL. Click on the link to see what happens.

In [9]:
print r.url

http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Duke+Ellington&api-key=ef9055ba947dd842effe0ecf5e338af9%3A15%3A72340235


Click on that link to see it returns!

### Challenge 1:  Adding a date range

What if we only want to search within a particular date range? The NYT Article Api allows us to specify start and end dates.

Alter the `search_params` code above so that the request only searches for articles in the year 2005.

You're gonna need to look at the documentation [here](http://developer.nytimes.com/docs/read/article_search_api_v2) to see how to do this.

In [10]:
#YOUR CODE HERE

# Uncomment to test
# r = requests.get(base_url+response_format, params=search_params)
# r.url

### Challenge 2:  Specifying a results page

The above will return the first 10 results. To get the next ten, you need to add a "page" parameter. Change the search parameters above to get the second 10 resuls. 

In [79]:
#YOUR CODE HERE

# Uncomment to test
# r = requests.get(base_url+response_format, params=search_params)
# r.url

## 2. Parsing the response text

We can read the content of the server’s response using `.text`

In [81]:
# Inspect the content of the response, parsing the result as text
response_text= r.text
print response_text[:1000]

{"response":{"meta":{"hits":6207,"time":38,"offset":0},"docs":[{"web_url":"http:\/\/www.nytimes.com\/2015\/08\/31\/movies\/gary-keys-filmmaker-who-documented-duke-ellington-dies-at-81.html","snippet":"Mr. Keys produced concerts at Carnegie Hall and Lincoln Center by Stan Getz, Dionne Warwick, the Supremes, Simon and Garfunkel, Judy Garland and Stevie Wonder.","lead_paragraph":"Mr. Keys produced concerts at Carnegie Hall and Lincoln Center by Stan Getz, Dionne Warwick, the Supremes, Simon and Garfunkel, Judy Garland and Stevie Wonder.","abstract":"Gary Keys, a documentary filmmaker who captured some of the most important figures in jazz from the 1960s through the 1980s, most notably Duke Ellington, dies at age 81.  ","print_page":"7","blog":[],"source":"The New York Times","multimedia":[{"width":190,"url":"images\/2015\/08\/31\/arts\/31KEYS-obit\/31KEYS-obit-thumbWide-v2.jpg","height":126,"subtype":"wide","legacy":{"wide":"images\/2015\/08\/31\/arts\/31KEYS-obit\/31KEYS-obit-thumbWide-v

What you see here is JSON text, encoded as unicode text. JSON stands for "Javascript object notation." It has a very similar structure to a python dictionary -- both are built on key/value pairs. This makes it easy to convert JSON response to a python dictionary.

In [82]:
# Convert JSON response to a dictionary
data=json.loads(response_text)
# data

That looks intimidating! But it's really just a big dictionary. Let's see what keys we got in there.

In [83]:
data.keys()

[u'status', u'response', u'copyright']

In [84]:
# this is boring
data['status']

u'OK'

In [124]:
# so is this
data['copyright']

u'Copyright (c) 2013 The New York Times Company.  All Rights Reserved.'

In [125]:
# this is what we want!
# data['response']

In [126]:
data['response'].keys()

[u'docs', u'meta']

In [127]:
data['response']['meta']['hits']

171

In [128]:
# data['response']['docs']
type(data['response']['docs'])

list

That looks what we want! Let's put that in it's own variable.

In [129]:
docs = data['response']['docs']

In [130]:
docs[0]

{u'_id': u'536415fa79881074d5bff10b',
 u'abstract': None,
 u'blog': [],
 u'byline': {u'contributor': u'',
  u'original': u'By ANITA GATES',
  u'person': [{u'firstname': u'Anita',
    u'lastname': u'GATES',
    u'organization': u'',
    u'rank': 1,
    u'role': u'reported'}]},
 u'document_type': u'article',
 u'headline': {u'main': u'Summer Movie Release Schedule',
  u'print_headline': u'Witches, Monsters and Men in Tights'},
 u'keywords': [{u'is_major': u'Y',
   u'name': u'subject',
   u'rank': u'1',
   u'value': u'Movies'}],
 u'lead_paragraph': u'Listings of this summer\u2019s releases.',
 u'multimedia': [],
 u'news_desk': u'Arts&Leisure',
 u'print_page': u'35',
 u'pub_date': u'2014-05-04T00:00:00Z',
 u'section_name': u'Movies',
 u'slideshow_credits': None,
 u'snippet': u'Listings of this summer\u2019s releases.',
 u'source': u'The New York Times',
 u'subsection_name': None,
 u'type_of_material': u'News',
 u'web_url': u'http://www.nytimes.com/2014/05/04/movies/summer-movie-release-sche

## 3. Putting everything together to get all the articles.

That's great. But we only have 10 items. The original response said we had 171 hits! Which means we have to make 171 /10, or 18 requests to get them all. Sounds like a job for a loop! 

But first, let's review what we've done so far.

In [11]:
# set key
key="ef9055ba947dd842effe0ecf5e338af9:15:72340235"

# set base url
base_url="http://api.nytimes.com/svc/search/v2/articlesearch"

# set response format
response_format=".json"

# set search parameters
search_params = {"q":"Duke Ellington",
                 "api-key":key,
                 "begin_date":"20140101", # date must be in YYYYMMDD format
                 "end_date":"20141231"}

# make request
r = requests.get(base_url+response_format, params=search_params)
    
# convert to a dictionary
data=json.loads(r.text)
    
# get number of hits
hits = data['response']['meta']['hits']
print "number of hits: " + str(hits)
    
# get number of pages
pages = int(math.ceil(hits/10))
    
# make an empty list where we'll hold all of our docs for every page
all_docs = [] 
    
# now we're ready to loop through the pages
for i in range(pages):
    print "collecting page " + str(i)
        
    # set the page parameter
    search_params['page'] = i
        
    # make request
    r = requests.get(base_url+response_format, params=search_params)
    
    # get text and convert to a dictionary
    data=json.loads(r.text)
        
    # get just the docs
    docs = data['response']['docs']
        
    # add those docs to the big list
    all_docs = all_docs + docs

number of hits: 171
collecting page 0
collecting page 1
collecting page 2
collecting page 3
collecting page 4
collecting page 5
collecting page 6
collecting page 7
collecting page 8
collecting page 9
collecting page 10
collecting page 11
collecting page 12
collecting page 13
collecting page 14
collecting page 15
collecting page 16
collecting page 17


In [12]:
len(all_docs)

171

### Challenge 3: Make a function

Turn the code above into a function that inputs a search term, and returns all the documents containing that search term in 2014.

In [121]:
#YOUR CODE HERE

## 4. Formatting and Exporting

Let's take another look at one of these documents.

In [143]:
all_docs[0]

{u'_id': u'5333318b79881069601eaa7f',
 u'abstract': None,
 u'blog': [],
 u'byline': {u'contributor': u'',
  u'original': u'By VIVIEN SCHWEITZER',
  u'person': [{u'firstname': u'Vivien',
    u'lastname': u'SCHWEITZER',
    u'organization': u'',
    u'rank': 1,
    u'role': u'reported'}]},
 u'document_type': u'article',
 u'headline': {u'content_kicker': u'Music Review',
  u'kicker': u'Music Review',
  u'main': u'Heavenly Voices Proclaim a Faith That Swings',
  u'print_headline': u'Heavenly Voices Proclaim a Faith That Swings',
  u'seo': u'Six Youth Choirs Sing From Ellington&#8217;s Sacred Concerts'},
 u'keywords': [{u'is_major': u'N',
   u'name': u'persons',
   u'rank': u'4',
   u'value': u'Ellington, Duke'},
  {u'is_major': u'N', u'name': u'subject', u'rank': u'2', u'value': u'Jazz'},
  {u'is_major': u'N', u'name': u'subject', u'rank': u'1', u'value': u'Music'},
  {u'is_major': u'N',
   u'name': u'organizations',
   u'rank': u'3',
   u'value': u'Carnegie Hall'}],
 u'lead_paragraph': u'

This is all great, but it's pretty messy. What we’d really like to to have, eventually, is a CSV, with each row representing an article, and each column representing something about that article (header, date, etc). As we saw before, the best way to do this is to make a lsit of dictionaries, with each dictionary representing an article and each dictionary representing a field of metadata from that article (e.g. headline, date, etc.) We can do this with a custom function:

In [149]:
def format_articles(unformatted_docs):
    '''
    This function takes in a list of documents returned by the NYT api 
    and parses the documents into a list of dictionaries, 
    with 'id', 'header', and 'date' keys
    '''
    formatted = []
    for i in unformatted_docs:
        dic = {}
        dic['id'] = i['_id']
        dic['headline'] = i['headline']['main'].encode("utf8")
        dic['date'] = i['pub_date'][0:10] # cutting time of day.
        formatted.append(dic)
    return(formatted) 

In [150]:
all_formatted = format_articles(all_docs)

In [152]:
all_formatted[:5]

[{'date': u'2014-03-27',
  'headline': 'Heavenly Voices Proclaim a Faith That Swings',
  'id': u'5333318b79881069601eaa7f'},
 {'date': u'2014-06-10',
  'headline': 'Next-Best Thing to Living Next Door to Your Idol',
  'id': u'5396048e79881005f88638af'},
 {'date': u'2014-02-17',
  'headline': 'Alice Babs, Who Sang for Ellington, Dies at 90',
  'id': u'52feeb1f7988103334201512'},
 {'date': u'2014-05-27',
  'headline': 'Herb Jeffries, \xe2\x80\x98Bronze Buckaroo\xe2\x80\x99 of Song and Screen, Dies at 100 (or So)',
  'id': u'5383698a79881002822dbd07'},
 {'date': u'2014-07-26',
  'headline': 'A Singer Blends Her Devotion to Jazz and the Baha\xe2\x80\x99i Faith',
  'id': u'53d2ee0f798810595081bc07'}]

### Challenge 4: Export the data to a CSV.

In [ ]:
#YOUR CODE HERE